# Generate Wikibase URIs

This notebook generates the wikibase URIs of the resources in a given ontology. It also serves as an example of how an ontology reasoner is used to infer whether an URI corresponds to a wikibase Item or Property.

In [7]:
import logging
import os
import sys

# set up module paths for imports
module_path = os.path.abspath(os.path.join('..'))
hercules_sync_path = os.path.abspath(os.path.join('..', 'hercules_sync'))
sys.path.append(module_path)
sys.path.append(hercules_sync_path)

# start logging system and set logging level
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting logger")

INFO:root:Starting logger


# Loading the ontology

In [2]:
ONTOLOGY_LOCATION = 'https://raw.githubusercontent.com/weso/hercules-ontology/master/src/asio-core.ttl'

In [3]:
from urllib.request import urlopen

onto_content = urlopen(ONTOLOGY_LOCATION).read().decode('utf-8')

In [4]:
from rdflib.graph import Graph

onto_graph = Graph().parse(format='turtle', data=onto_content)

## Obtaining the classes and properties

In [6]:
import ontospy

model = ontospy.Ontospy(data=onto_content, verbose=True, rdf_format='ttl')

----------
Reading: '@prefix :  ...'
.. trying rdf serialization: <ttl>
..... success!
----------
Loaded 1341 triples.
----------
RDF sources loaded successfully: 1 of 1.
..... 'Data: '@prefix :  ...''
----------


Scanning entities...
----------
Ontologies.........: 1
Classes............: 71
Properties.........: 48
..annotation.......: 3
..datatype.........: 6
..object...........: 39
Concepts (SKOS)....: 0
Shapes (SHACL).....: 0
----------


In [6]:
model.all_classes[:5]

[<Class *http://www.asio.es/asioontologies/asio#AcademicDegree*>,
 <Class *http://www.asio.es/asioontologies/asio#AcademicPublication*>,
 <Class *http://www.asio.es/asioontologies/asio#AdministrativeEntity*>,
 <Class *http://www.asio.es/asioontologies/asio#AdministrativePersonnel*>,
 <Class *http://www.asio.es/asioontologies/asio#Article*>]

In [16]:
model.all_properties_object

[<Property *http://www.asio.es/asioontologies/asio#Publishes*>,
 <Property *http://www.asio.es/asioontologies/asio#addresses*>,
 <Property *http://www.asio.es/asioontologies/asio#advises*>,
 <Property *http://www.asio.es/asioontologies/asio#atContext*>,
 <Property *http://www.asio.es/asioontologies/asio#authors*>,
 <Property *http://www.asio.es/asioontologies/asio#capital*>,
 <Property *http://www.asio.es/asioontologies/asio#codeOf*>,
 <Property *http://www.asio.es/asioontologies/asio#country*>,
 <Property *http://www.asio.es/asioontologies/asio#employs*>,
 <Property *http://www.asio.es/asioontologies/asio#funds*>,
 <Property *http://www.asio.es/asioontologies/asio#geo-division*>,
 <Property *http://www.asio.es/asioontologies/asio#hasCV*>,
 <Property *http://www.asio.es/asioontologies/asio#hasCode*>,
 <Property *http://www.asio.es/asioontologies/asio#hasPatent*>,
 <Property *http://www.asio.es/asioontologies/asio#hasResearchContribution*>,
 <Property *http://www.asio.es/asioontologies/

In [14]:
model.all_properties[11].__class__

ontospy.core.entities.OntoProperty

In [65]:
type(model.all_properties)

list

In [62]:
uri_items = map(lambda x: x.uri, model.all_classes)
uri_props = map(lambda x: x.uri, model.all_properties)

In [63]:
triple = None
for el in onto_graph:
    triple = el

In [64]:
triple[0] in uri_props

True

In [52]:
triple[0]

rdflib.term.URIRef('http://www.asio.es/asioontologies/asio#funds')

## Creating ids for each item and property

In [25]:
class WBIdGeneratorMock():
    def __init__(self):
        self.q_id = 0
        self.p_id = 0
    
    def generate_item_id(self):
        self.q_id += 1
        return f"Q{self.q_id}"
    
    def generate_prop_id(self):
        self.p_id += 1
        return f"P{self.p_id}"


In [24]:
from hercules_sync.external.uri_factory_mock import URIFactory

factory = URIFactory()
factory.reset_factory()

In [ ]:
id_generator = WBIdGeneratorMock()

for item in uri_items:
    item_id = uri_generator.generate_item_id()
    factory.post_uri(item, new_id)

for item in uri_props:
    prop_id = uri_generator.generate_prop_id()
    factory.post_uri(item, prop_id)

## Save the mappings

In [ ]:
all_uris = uri_items + uri_props

